In [1]:
import blackjack_engine
import random
import numpy as np
import pandas as pd

In [67]:
# from blackjack_api_example.py
def padarray(A, size):
    if len(A) == 0:
        A = ['']
    t = size - len(A)
    arr = np.pad(A, pad_width=(0, t), mode='empty')
    
    return arr.astype('<U32')

def player_bot(hand):
    """ makes random choice, or makes decisions using neural net etc"""
    # decision making code goes here
    return random.choice(['hit', 'stay']) 

def play_blackjack_game():
    # initialize a game with shuffled deck and hands dealt to dealer and player:
    game = blackjack_engine.BlackjackGame()
    moves = []
    #game_states = []
    dealer_cards = []
    player_hands = []
    game_results = []
    if game.is_finished == False: # if nobody got dealt a blackjack, loop through decisions
        while True: 
            player_hand = game.player_hands[0].cards[:]
            dealer_card = game.dealer_hand.cards[0]
            #game_state_row = {'player_hand':player_hand,'dealer_card':dealer_card}
            #game_state_row = [player_hand, dealer_card]
            player_hands.append(player_hand)
            dealer_cards.append(dealer_card)
            hit_or_stay = player_bot(game.player_hands)
            moves.append(hit_or_stay)
            game.player_move(hit_or_stay)

            if not game.is_finished:
                continue # game is not finished so continue loop
            if game.is_finished:
                [game_results.append(game.result) for game_result in np.arange(len(moves))]
                break # game is finished so exit loop
    return player_hands, dealer_cards, moves, game_results

# implement a training set generation routine: give the API random actions and see how games turn out
def build_training_set(n_pts):
    '''
    state_of_play is an n x m array of values where n is the number of data points, and m is the max number of features in a hand 
    y is an n x 1 array, where n is the number of data points (number of times the player drew a card, basically) 
    '''
    all_player_hands = []
    all_dealer_cards = []
    all_moves = []
    all_game_results = []

    for pt in np.linspace(0,n_pts,n_pts+1):
        [player_hands, dealer_cards, moves,game_results] = play_blackjack_game()
        all_player_hands.append(player_hands)
        all_dealer_cards.append(dealer_cards)
        all_moves.append(moves)
        all_game_results.append(game_results)
    return all_player_hands, all_dealer_cards, all_moves, all_game_results

In [70]:
[all_player_hands, all_dealer_cards, all_moves, all_game_results] = build_training_set(10)
all_player_hands


[[['Q', '2'], ['Q', '2', '4']],
 [['4', '6'], ['4', '6', '5'], ['4', '6', '5', 'A']],
 [['7', '7']],
 [],
 [['Q', '10']],
 [['6', '8']],
 [['Q', '5']],
 [['10', '8']],
 [['A', 'A']],
 [['7', 'K']],
 [['3', '6']]]

In [71]:
all_moves

[['hit', 'hit'],
 ['hit', 'hit', 'hit'],
 ['stay'],
 [],
 ['hit'],
 ['hit'],
 ['stay'],
 ['stay'],
 ['stay'],
 ['stay'],
 ['stay']]

In [78]:
# Flatten the data such that each row represents a decision instance
flat_moves = list(np.concatenate(all_moves).flat)
#flat_player_hands = list(np.concatenate(all_player_hands).flat)
flat_game_results = list(np.concatenate(all_game_results).flat)
all_player_hands

[[['Q', '2'], ['Q', '2', '4']],
 [['4', '6'], ['4', '6', '5'], ['4', '6', '5', 'A']],
 [['7', '7']],
 [],
 [['Q', '10']],
 [['6', '8']],
 [['Q', '5']],
 [['10', '8']],
 [['A', 'A']],
 [['7', 'K']],
 [['3', '6']]]

In [50]:
# build the input and output datasets
# input: game_states (padded), results 
# output: decision (hit or stay)

In [51]:
# train the neural network on the previously generated data
def build_model(X,y):
    input_shape = np.shape(X)
    output_shape = np.shape(y) 
    # use a simple sequential dense model to predict which actions will result in a win
    return model

def train_model(untrained_model,training_data):
    training_data = [X_train, y_train, X_test, y_test]
    
    return trained_network

def use_model_to_make_decision(trained_model,state_of_play):

    decision = trained_model.predict(state_of_play)
    while game_is_still_in_play:
        state_of_play = play_blackjack_hand(state_of_play,decision)
        game_is_still_in_play = check_result(state_of_play)
    return state_of_play
    

In [52]:
flat_game_states[0]

{'player_hand': ['6', 'K'], 'dealer_card': '8'}

In [32]:
flat_game_results

[-1.0, -1.0, -1.0, -1.0, -1.0, 0.0, -1.0, -1.0, -1.0, 0.0, 0.0, -1.0, -1.0]